Load Libraries

In [ ]:
#### Load Libraries
import numpy as np
from joblib import load
from keras.models import Model, load_model
import glob 
import numpy
# import pandas
import  pandas as pd
import matplotlib.pyplot as plt
from keras.losses import mean_absolute_error as mae, mean_squared_error as mse, mean_absolute_percentage_error as mape
import os
from pymongo import MongoClient
import plotly.express as ex
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime
from datetime import timedelta
import plotly.io as pio

DB Config

In [ ]:
#####DB Config
#####DB Config
hh=5
host= "m53763edge"
port=27017
collection= "thermal_data"
all_data= True
get_preprocessed_data= False

Set parameters

In [ ]:
version= 'v0.0.4'
workspace= "G:\\Innovations@HELLER\\DN\\KI\\Zollern-FH-MillTrunMotor\\Datasets\\workspace\\data\\training"
assests_dir= 'G:\\Innovations@HELLER\\DN\\KI\\Zollern-FH-MillTrunMotor\\Datasets\\workspace\\assets\\'+version+'\\'
preprocessor_dir=assests_dir
preprocessor_name='preprocessor.p'
model_dir= assests_dir
model_name= 'model.h5'
###['Drehzahl','Temp_Motor','Temp_Klemmung', 'Temp_Lager']
model_input_Columns=  ['T_MOTOR','T_KLEMMUNG', 'T_LAGER' ]#'DRZ5',
output_variable= 'prediction'
prediction_variable= 'prediction'
window=10
shift= 1
time_shift= +2 

Set time priod needed

In [ ]:
start= datetime.strptime("2023-06-01T14:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
end= datetime.strptime ("2023-06-02T13:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")


Get Data from database

In [ ]:
## Read raw and preprocessing data
client = MongoClient(host= host, port=port)
db = client.h4ai
event_list = db[collection].find({ "date" : { '$gte' : start, '$lt' : end} }).sort('date', 1)
signals= None
i=0
last_prediction= -500
prev_time= None
current_time= None
stop= False
### Go through events in DB
for event in event_list:
    i+=1
    record_list= event['content']
    keys= None
    ### Go through records in each event
    for record in record_list:
        ### Get right data between start / end 
        if record['date']< start or record['date'] > end:
            continue
        current_time= record['date']+ timedelta(hours= time_shift)
        #current_time= current_time.replace(microsecond=0)
        if prev_time is not None  and (current_time -prev_time).total_seconds()> 5.5:
            print('Period= ', (current_time -prev_time).total_seconds())
            print('prev_time', prev_time)
            print('current_time', current_time)
            print('#########################################################')
        for item in record['raw_data'].keys():
            record['raw_data'][item]= [np.double(np.round(record['raw_data'][item],decimals=2))]
        record['raw_data']['date']=[current_time]
        record['raw_data']['given2model']=[record['given2model']]
        record['raw_data']['prediction']= [record['prediction']]
        ####################################
        #print ('After ',record)
        lf_signal_point= pd.DataFrame(record['raw_data'])
        if signals is None:
            signals= lf_signal_point
        else:
            signals= signals.append(lf_signal_point,ignore_index= True)
        prev_time= current_time
signals.reset_index(inplace= True)
output=  signals[output_variable]
machine_prediction= signals[prediction_variable]

Read Magent Temperature File

In [ ]:
file_path= "G:\\Innovations@HELLER\\DN\\KI\\Zollern-FH-MillTrunMotor\\Datasets\\rough_data\\Rotor_temperatur.csv"
df= pd.read_csv(filepath_or_buffer=file_path,infer_datetime_format= True, parse_dates= True)
df['date']= pd.to_datetime(df['date'])+ pd.DateOffset(hours=+1)

Visualize the retreived data

In [ ]:
scatter_mode= 'lines'#'lines'# 'lines+markers'# 'markers'
selected_Columns= ['DRZ','Temp_Klemmung', 'Temp_Lager','Temp_Motor','Analytic_sol', prediction_variable, output_variable, 'db_prediction_abs_error']
y_axis_names= selected_Columns
#print(preds)
fig= make_subplots(rows=5,cols=1,shared_xaxes= True, print_grid= True, vertical_spacing=0.02)
##DRZ
fig.add_trace(go.Scatter(x= signals['date'], y= signals['DRZ5'], name='given2model', mode= scatter_mode), row= 1, col= 1)
fig.update_yaxes(title_text= 'DRZ (RPM)', row= 1, col= 1)
##T_KLEMMUNG
fig.add_trace(go.Scatter(x= signals['date'], y= signals['T_KLEMMUNG'], name='T_KLEMMUNG', mode= scatter_mode), row= 2, col= 1)
fig.update_yaxes(title_text= 'T_KLEMMUNG C°', row= 2, col= 1)
##T_LAGER
fig.add_trace(go.Scatter(x= signals['date'], y= signals['T_LAGER'], name='T_LAGER', mode= scatter_mode), row= 3, col= 1)
fig.update_yaxes(title_text= 'T_LAGER C°', row= 3, col= 1)
##'T_MOTOR'
fig.add_trace(go.Scatter(x= signals['date'], y= signals['T_MOTOR'], name='T_MOTOR', mode= scatter_mode), row= 4, col= 1)
fig.update_yaxes(title_text= 'T_MOTOR C°', row= 4, col= 1)
##'prediction_variable'
fig.add_trace(go.Scatter(x= df['date'], y= df['magnet_temperature'], name='magnet_temperature', mode= 'markers'), row= 5, col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= signals[prediction_variable], name=prediction_variable, mode= scatter_mode), row= 5, col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= signals['Analytic_sol'], name='Analytic_sol', mode= 'markers'), row= 5, col= 1)
fig.update_yaxes(title_text= 'Rotor Temperatur C°', row= 5, col= 1)

fig.update_layout(height=1200, width=1400, title_text= 'M57002 Machine Data')
fig.show()

In [ ]:
#df['date']= pd.to_datetime(df['date'])+ pd.DateOffset(seconds=-2)

In [ ]:
df.date

In [ ]:
sync_point_start= '2023-06-05 09:00:00.00'

In [ ]:
df_db= signals[signals['date']>= sync_point_start] #signals[< sync_point_end]
df_db

In [ ]:
df_sensor= df[df['date']>= sync_point_start]
df_sensor.reset_index(inplace= True)
df_sensor

In [ ]:
min_len= len(df_sensor)

In [ ]:
df_db= df_db.iloc[0:min_len]
df_db.reset_index(inplace= True)
print(len(df_db))

In [ ]:
path= 'G:\\Innovations@HELLER\\DN\\KI\\Zollern-FH-MillTrunMotor\\Datasets\\rough_data\\database_with_sensor\\{file_name}'

In [ ]:
df_sensor['magnet_temperature']

In [ ]:
df_db= df_db.join(df_sensor['magnet_temperature'])

df_db.to_csv(path.format(file_name= 'Versuch_05062023.csv'))